In [1]:
import pandas as pd
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split

iris = load_iris()
X = pd.DataFrame(iris['data'], columns=iris['feature_names'])
y = pd.Series(iris['target'], name='class')
X["y"] = y
X.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'y'],
      dtype='object')

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train["y"] = y_train

<ipython-input-2-0ea5186d8679>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["y"] = y_train


In [3]:
def separa_grupos(coluna, valor_corte, grupo):
    R, L = [], []
    idx = 0
    while idx < len(grupo):
        amostra = grupo[idx]
        if amostra[coluna].item() < valor_corte:
            L.append(amostra)
        else:
            R.append(amostra)
        idx += 1
    return L,R
print(X.columns)
print("\nSeparando os grupos com base na coluna sepal length (0) no valor 4,5.")
L, R = separa_grupos(0, 4.5, np.array(X_train))
L

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'y'],
      dtype='object')

Separando os grupos com base na coluna sepal length (0) no valor 4,5.


[array([4.4, 2.9, 1.4, 0.2, 0. ]),
 array([4.3, 3. , 1.1, 0.1, 0. ]),
 array([4.4, 3. , 1.3, 0.2, 0. ])]

In [4]:
def conta_valores(grupo):
    dic = {}
    apareceu = []
    for amostra in grupo:
        if amostra[-1] not in apareceu:
            dic[amostra[-1]] = 1
            apareceu.append(amostra[-1])
        else:
            dic[amostra[-1]] += 1
    return dic

conta_valores(np.array(X))

{0.0: 50, 1.0: 50, 2.0: 50}

In [5]:
def calcula_gini(grupo):
    dic_gini = conta_valores(grupo)

    gini = 1
    for v in dic_gini.values():
        gini -= (v/sum(dic_gini.values()))**2
    return gini

gini_dataset = calcula_gini(np.array(X_train))
print("Gini do X_train:", gini_dataset)
L, R = separa_grupos(0, 4.5, np.array(X_train))
gini_L = calcula_gini(np.array(L))
print("Gini da partição:", gini_L)

Gini do X_train: 0.6658000000000001
Gini da partição: 0.0


In [6]:
def acha_ganho(L, R, incerteza_atual):
    p = float(len(L)) / (len(L) + len(R))
    return incerteza_atual - p * calcula_gini(L) - (1 - p) * calcula_gini(R)

L, R = separa_grupos(0, 4.5, np.array(X_train))
print("Ganho para partição de sepal length (cm) no valor 4,5: {}".format(acha_ganho(L, R, gini_dataset))) 
L, R = separa_grupos(0, 5, np.array(X_train))
print("Ganho para partição de sepal length (cm) no valor 5: {}".format(acha_ganho(L, R, gini_dataset))) 
L, R = separa_grupos(0, 5.5, np.array(X_train))
print("Ganho para partição de sepal length (cm) no valor 5,5: {}".format(acha_ganho(L, R, gini_dataset)))

Ganho para partição de sepal length (cm) no valor 4,5: 0.02208865979381458
Ganho para partição de sepal length (cm) no valor 5: 0.07310897009966788
Ganho para partição de sepal length (cm) no valor 5,5: 0.211876388888889


In [7]:
def acha_melhor_corte(grupo):
            melhor_ganho = 0
            melhor_col = None
            melhor_val = None
            incerteza_atual = calcula_gini(grupo)
            n_features = len(grupo[0]) - 1

            for col in range(n_features):

                valores = set([linha[col] for linha in grupo])

                for val in valores:

                    l, r = separa_grupos(col,val,grupo)

                    if len(l) == 0 or len(r) == 0:
                        continue

                    ganho = acha_ganho(l, r, incerteza_atual)

                    if ganho >= melhor_ganho:
                        melhor_ganho, melhor_col, melhor_val = ganho, col, val

            return melhor_ganho, melhor_col, melhor_val

m_g, m_c, m_v = acha_melhor_corte(np.array(X_train))
print("Melhor ganho: {0}, Melhor coluna para particionar {1} e melhor valor para particionar {2}".format(m_g,m_c,m_v))
Left, Right = separa_grupos(3, 1.0, np.array(X_train))
print("Ganho para partição de petal width (cm) no valor 1: {}".format(acha_ganho(Left, Right, gini_dataset)))
Left, Right = separa_grupos(3, 1.5, np.array(X_train))
print("Ganho para partição de petal width (cm) no valor 1.5: {}".format(acha_ganho(Left, Right, gini_dataset)))
Left, Right = separa_grupos(3, 0.5, np.array(X_train))
print("Ganho para partição de petal width (cm) no valor 0.5: {}".format(acha_ganho(Left, Right, gini_dataset))) 

Melhor ganho: 0.320872463768116, Melhor coluna para particionar 3 e melhor valor para particionar 1.0
Ganho para partição de petal width (cm) no valor 1: 0.320872463768116
Ganho para partição de petal width (cm) no valor 1.5: 0.2267852216748768
Ganho para partição de petal width (cm) no valor 0.5: 0.29171549295774657


In [8]:
class Folha:
    def __init__(self, grupo):
        self.predicoes = conta_valores(grupo)
        
class No:
    def __init__(self, coluna, valor, right, left):
        self.coluna = coluna
        self.valor  = valor
        self.right  = right
        self.left   = left

In [9]:
def construir_arvore(grupo):
    ganho, coluna, valor = acha_melhor_corte(grupo)

    if ganho == 0:
        return Folha(grupo)

    left, right  = separa_grupos(coluna, valor, grupo)

    right_branch = construir_arvore(right)
    left_branch = construir_arvore(left)

    return No(coluna, valor, right_branch, left_branch)


def desenha_arvore(no, espaco=""):
    if isinstance(no, Folha):
        print (espaco + "Predict", no.predicoes)
        return

    print (espaco + str(no.coluna) + " " + str(no.valor))

    print (espaco + '--> Right:')
    desenha_arvore(no.right, espaco + "  ")

    print (espaco + '--> Left:')
    desenha_arvore(no.left, espaco + "  ")

arvore = construir_arvore(np.array(X_train))
desenha_arvore(arvore)

3 1.0
--> Right:
  3 1.8
  --> Right:
    2 4.9
    --> Right:
      Predict {2.0: 28}
    --> Left:
      1 3.2
      --> Right:
        Predict {1.0: 1}
      --> Left:
        Predict {2.0: 2}
  --> Left:
    2 5.6
    --> Right:
      Predict {2.0: 2}
    --> Left:
      0 5.0
      --> Right:
        1 2.3
        --> Right:
          Predict {1.0: 31}
        --> Left:
          3 1.5
          --> Right:
            Predict {2.0: 1}
          --> Left:
            Predict {1.0: 2}
      --> Left:
        3 1.7
        --> Right:
          Predict {2.0: 1}
        --> Left:
          Predict {1.0: 1}
--> Left:
  Predict {0.0: 31}


In [10]:
def preditor(no, amostra):

    if isinstance(no, Folha):
        return no.predicoes

    if amostra[no.coluna] >= no.valor:
        return preditor(no.right ,amostra)
    else:
        return preditor(no.left, amostra)

p = preditor(arvore, np.array(X_train)[42])
print(p)
print(np.array(X_train)[42])

{1.0: 31}
[6.  2.7 5.1 1.6 1. ]


In [11]:
def desenha_folha(predicao):
    total = sum(predicao.values()) * 1.0
    probs = {}
    for lbl in predicao.keys():
        probs[lbl] = str(int(predicao[lbl] / total * 100)) + "%"
    return probs
desenha_folha(p)

{1.0: '100%'}

In [12]:
X_test["y"] = y_test
for amostra in np.array(X_test):
    print ("Real: {} Predito: {}".format(amostra[-1], desenha_folha(preditor(arvore, amostra))))

Real: 1.0 Predito: {1.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 1.0 Predito: {1.0: '100%'}
Real: 1.0 Predito: {1.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 1.0 Predito: {1.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 1.0 Predito: {2.0: '100%'}
Real: 1.0 Predito: {1.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 1.0 Predito: {1.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 1.0 Predito: {1.0: '100%'}
Real: 1.0 Predito: {1.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 2.0 Predito: {2.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 0.0 Predito: {0.0: '100%'}
Real: 0.0 

<ipython-input-12-32d2de6d6979>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["y"] = y_test
